In [1]:
import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from scipy.linalg import expm
from scipy.sparse import csr_matrix
import pickle
from copy import deepcopy
from qiskit.quantum_info import DensityMatrix, Statevector, partial_trace, state_fidelity
import time

from oft_n_liouv_np import oft
from tools.classical import find_ideal_heisenberg, trotter_step_heisenberg_qt, ham_evol_qt, hamiltonian_matrix

np.random.seed(667)
np.set_printoptions(precision=10, suppress=False)
#TODO: Compare rounded OFT with exact OFT later, to see how much rounding is still fine!
#TODO: State is rho not a pure state psi...

In [2]:
num_qubits = 2
num_energy_bits = 6
oft_precision = int(np.ceil(np.abs(np.log10(2**(-num_energy_bits)))))
print(oft_precision)
delta = 0.01
eps = 0.1
sigma = 5
bohr_bound = 0 #2 ** (-num_energy_bits + 1)
beta = 1
eig_index = 2
jump_site_index = 0

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, [1, 1, 1, 1], signed=False, for_oft=True)
initial_state = hamiltonian.eigvecs[:, eig_index]

N = 2**num_energy_bits
N_labels = np.arange(N / 2, dtype=int)
N_labels_neg = np.arange(- N / 2, 0, dtype=int)
N_labels = np.concatenate((N_labels, N_labels_neg))
energy_labels = 2 * np.pi * N_labels / N 
time_labels = N_labels

# rand_jump_index = np.random.randint(0, num_qubits)ú
site_list = [qt.qeye(2) for _ in range(num_qubits)]
x_jump_ops = []
for q in range(num_qubits):
    site_list[q] = qt.sigmax()
    x_jump_ops.append(qt.tensor(site_list).full())

x_jump = x_jump_ops[jump_site_index]

2
Original spectrum:  [-6.  2.  2.  2.]
Ideal spectrum:  [0.   0.45 0.45 0.45]
Nonrescaled coefficients:  [1, 1, 1, 1]
Rescaled coefficients:  [0.05625 0.05625 0.05625 0.05625]


### Energy decompositions of jump op A and initial states

In [3]:
# energy_projectors = [qt.ket2dm(qt.Qobj(energy_eigenstates[i])) for i in range(2**num_qubits)]

# Write jump operator in energy eigenbasis, A' = U^dag A U
x_jump_in_eigenbasis = hamiltonian.eigvecs.conj().T @ x_jump_ops[0] @ hamiltonian.eigvecs
print(x_jump)
print('Jump in eigenbasis:')
print(x_jump_in_eigenbasis)
    

[[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]]
Jump in eigenbasis:
[[ 0.          +0.j -0.7071067812+0.j  0.          +0.j  0.7071067812+0.j]
 [-0.7071067812+0.j  0.          +0.j  0.7071067812+0.j  0.          +0.j]
 [ 0.          +0.j  0.7071067812+0.j  0.          +0.j  0.7071067812+0.j]
 [ 0.7071067812+0.j  0.          +0.j  0.7071067812+0.j  0.          +0.j]]


In [4]:
# Check by constructing jump operator that brings system to eigenstate 0
jump_to_0 = hamiltonian.eigvecs[:, 0] @ hamiltonian.eigvecs[:, eig_index].conj().T
print(jump_to_0)
jump_to_0_in_eigenbasis = hamiltonian.eigvecs.conj().T @ jump_to_0 @ hamiltonian.eigvecs  #* This for jump operators
print(jump_to_0_in_eigenbasis)

[[ 0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0.5+0.j  0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j -0.5+0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j]]
[[6.2232853217e-19+0.j 0.0000000000e+00+0.j 1.0000000000e+00+0.j
  0.0000000000e+00+0.j]
 [0.0000000000e+00+0.j 0.0000000000e+00+0.j 0.0000000000e+00+0.j
  0.0000000000e+00+0.j]
 [5.9779208748e-34+0.j 0.0000000000e+00+0.j 3.2517679528e-17+0.j
  0.0000000000e+00+0.j]
 [0.0000000000e+00+0.j 0.0000000000e+00+0.j 0.0000000000e+00+0.j
  0.0000000000e+00+0.j]]


In [5]:
# Find decomposition of a vector in the energy eigenbasis
vec = (hamiltonian.eigvecs[:, 0] + hamiltonian.eigvecs[:, 2]) / np.sqrt(2)
rho = vec @ vec.conj().T
vec_eigenbasis_coeffs = hamiltonian.eigvecs.conj().T @ vec  #* This for states
print(vec_eigenbasis_coeffs)
rho_eigenbasis = hamiltonian.eigvecs.conj().T @ rho @ hamiltonian.eigvecs
print(rho_eigenbasis)


[[0.7071067812+0.j]
 [0.          +0.j]
 [0.7071067812+0.j]
 [0.          +0.j]]
[[0.5+0.j 0. +0.j 0.5+0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0.5+0.j 0. +0.j 0.5+0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]]


In [6]:
# Define your function that takes a single matrix entry as input
def transform_function(entry):
    # Example transformation: square the input
    return entry ** 2

def gaussian(entry):
    trafod_entry = np.exp(-((0.16 - entry) ** 2) * sigma ** 2)
    return trafod_entry

# Example numpy matrix
input_matrix = np.array([[1, 2, 3],
                         [4, 5, 6],
                         [7, 8, 9]])

# Vectorize the function (if it's not already vectorized)
vectorized_function = np.vectorize(transform_function)
vec_gaussian_fn = np.vectorize(gaussian)

# Apply the function to each entry in the matrix
output_matrix = vectorized_function(input_matrix)
output_matrix_gaussian = vec_gaussian_fn(input_matrix)

print(output_matrix)
print(output_matrix_gaussian)
matrix1 = np.array([[1, 2, 3],
                    [4, 5, 6],
                    [7, 8, 9]])

matrix2 = np.array([[9, 8, 7],
                    [6, 5, 4],
                    [3, 2, 1]])

# Perform element-wise multiplication
result_matrix = matrix1 * matrix2

print(result_matrix)

[[ 1  4  9]
 [16 25 36]
 [49 64 81]]
[[2.1829577951e-008 1.7430708966e-037 2.6844830678e-088]
 [7.9741094286e-161 4.5685630002e-255 0.0000000000e+000]
 [0.0000000000e+000 0.0000000000e+000 0.0000000000e+000]]
[[ 9 16 21]
 [24 25 24]
 [21 16  9]]


### Precompute all Bohr frequencies and just use them later

In [15]:
#TODO: The new idea is to just take the gaussian weighed bohr freq matrix and multiply entry by entry with jump in eigenbasis
# This multiplies coeffs of the matrix with the gaussian weights, and then we transform back to computational basis and done I think.
#* Scales in time almost exactly as diagonalization.

phase = -0.44 * N / (2 * np.pi)
print(phase)
print(f'Energy = {2 * np.pi * phase / N}')

def gaussian_weight(v, phase, N, sigma_t): #! Don't forget normalizations
    energy = 2 * np.pi * phase / N
    return np.exp(-(energy - v) ** 2 * sigma_t ** 2)

add_gaussian_weight = np.vectorize(gaussian_weight)

jump_op = x_jump
# print(jump_op)
oft_precision = int(np.ceil(np.abs(np.log10(N**(-1)))))
    
bohr_freqs = hamiltonian.spectrum[:, np.newaxis] - hamiltonian.spectrum
# print('Bohr freqs:')
# print(bohr_freqs)
t0 = time.time()
oft_jump = np.einsum('ij, jk, jm, mn -> in', 
                    hamiltonian.eigvecs.conj().T,
                    add_gaussian_weight(bohr_freqs, phase, N, sigma),
                    hamiltonian.eigvecs.conj().T,
                    jump_op, optimize=True)

tt = time.time()
print(f'OFT time = {tt - t0}')
gaussian_bohr_freqs = add_gaussian_weight(bohr_freqs, phase, N, sigma)
tweighing = time.time()
# print('Gaussian freqs:')
# print(gaussian_bohr_freqs)
jump_in_eigenbasis = hamiltonian.eigvecs.conj().T @ jump_op @ hamiltonian.eigvecs
# print(jump_in_eigenbasis)

# weighted_jump_in_eigenbasis = gaussian_bohr_freqs * jump_in_eigenbasis
# oft_jump = hamiltonian.eigvecs @ weighted_jump_in_eigenbasis @ hamiltonian.eigvecs.conj().T

jump_nonzero_indices = np.nonzero(jump_in_eigenbasis)
nonzero_index_pairs = list(zip(*jump_nonzero_indices))

# For now we take energy diffs close to each other as the same energy diff (cuts off a ton of sum terms)
bohr_freqs_of_jump = np.round(bohr_freqs[jump_nonzero_indices], oft_precision+2)
print(bohr_freqs_of_jump)

uniqe_freqs = np.unique(bohr_freqs_of_jump, return_index=True)
print(uniqe_freqs)
gauss_weighted_unique_freqs = add_gaussian_weight(uniqe_freqs[0], phase, N, sigma)
print(gauss_weighted_unique_freqs)
same_freq_indices_in_energy_list = [np.nonzero(bohr_freqs_of_jump == bohr_freqs_of_jump[uniqe_freqs[1][i]]) 
                                for i in range(len(uniqe_freqs[0]))]

-4.481803197467773
Energy = -0.44
OFT time = 0.0003268718719482422
[-0.45 -0.45  0.45  0.    0.    0.    0.45  0.  ]
(array([-0.45,  0.  ,  0.45]), array([0, 3, 2]))
[9.9750312240e-01 7.9070540516e-03 2.5112128333e-09]


In [8]:
bohr_freqs = hamiltonian.spectrum[:, np.newaxis] - hamiltonian.spectrum

print(bohr_freqs)

[[ 0.   -0.45 -0.45 -0.45]
 [ 0.45  0.    0.    0.  ]
 [ 0.45  0.    0.    0.  ]
 [ 0.45  0.    0.    0.  ]]


In [9]:
hamiltonian.spectrum[:, np.newaxis]

array([[6.6613381478e-17],
       [4.5000000000e-01],
       [4.5000000000e-01],
       [4.5000000000e-01]])

### Find all possible energy jumps for a give state and jump operator A

Jump operator in the eigenbasis shows at each entry which eigenstate jumps are possible, so for each nonzero entry we can take their indices and find the possible energy differences. But of course also depending on the initial state some of these will or will not happen.

In [10]:
# Indices of nonzero entries in jump operator
jump_nonzero_indices = np.nonzero(x_jump_in_eigenbasis)
print(jump_nonzero_indices)
nonzero_index_pairs = list(zip(*jump_nonzero_indices))
print(nonzero_index_pairs)
# Convert list of tuples back to tuple of arrays
# jump_nonzero_indices = tuple(np.array(i) for i in zip(*nonzero_index_pairs))
# print(jump_nonzero_indices)

(array([0, 0, 1, 1, 2, 2, 3, 3]), array([1, 3, 0, 2, 1, 3, 0, 2]))
[(0, 1), (0, 3), (1, 0), (1, 2), (2, 1), (2, 3), (3, 0), (3, 2)]


In [11]:
# Get all Bohr frequencies for the nonzero jump op indices
# bohr_freqs_for_jump = bohr_freqs[jump_nonzero_indices]
bohr_freqs_for_jump = np.round(bohr_freqs[jump_nonzero_indices], oft_precision+2)
print('These are all the energy jumps in the system:')
print(bohr_freqs_for_jump)

# Bohr frequencies that are unique but up to some precision:
# uniqe_freqs = np.unique(bohr_freqs_for_jump, return_index=True)
uniqe_freqs = np.unique(np.round(bohr_freqs_for_jump, oft_precision+2), return_index=True)  #! Change this rounding if needed
print(f'Number of unique frequencies: {len(uniqe_freqs[0])}')
# Get the indices of the same Bohr frequencies
same_freq_indices_in_energy_list = [np.nonzero(bohr_freqs_for_jump == bohr_freqs_for_jump[uniqe_freqs[1][i]]) 
                                    for i in range(len(uniqe_freqs[0]))]

print(f'Unique Bohr Frequencies: {uniqe_freqs[0]}')
print(f'Indices of the same Bohr Frequencies in the list of energy diffs: {same_freq_indices_in_energy_list}')
same_freq_indices_of_jump_op = []  #* This is what I will need in the main computation multiplied by the states coeffs (in eigenbasis) of the latter index
for v in range(len(same_freq_indices_in_energy_list)):
    same_freq_indices_of_jump_op.append([])
    for index in same_freq_indices_in_energy_list[v][0]:
        same_freq_indices_of_jump_op[v].append(nonzero_index_pairs[index])
        
print(f'Indices of the same Bohr Frequencies in the jump operator: {same_freq_indices_of_jump_op}')

These are all the energy jumps in the system:
[-0.45 -0.45  0.45  0.    0.    0.    0.45  0.  ]
Number of unique frequencies: 3
Unique Bohr Frequencies: [-0.45  0.    0.45]
Indices of the same Bohr Frequencies in the list of energy diffs: [(array([0, 1]),), (array([3, 4, 5, 7]),), (array([2, 6]),)]
Indices of the same Bohr Frequencies in the jump operator: [[(0, 1), (0, 3)], [(1, 2), (2, 1), (2, 3), (3, 2)], [(1, 0), (3, 0)]]


In [12]:
# Construct jump operators for each Bohr frequency (not necessary for main computation but good for checking)

#! Dude when you do this part just multiply with gaussian factor at the right nű and it's done, that's A(w) if it's summed up

print(x_jump_in_eigenbasis)
jump_ops_for_energy = []
for energy_index in range(len(same_freq_indices_of_jump_op)):
    index_tuples_for_energy = same_freq_indices_of_jump_op[energy_index]
    jump_op_indices_for_energy = tuple(np.array(i) for i in zip(*index_tuples_for_energy))
    jump_op_for_energy = np.zeros((2**num_qubits, 2**num_qubits), dtype=complex)
    jump_op_for_energy[jump_op_indices_for_energy] = x_jump_in_eigenbasis[jump_op_indices_for_energy]
    jump_ops_for_energy.append(jump_op_for_energy)

for v in range(len(jump_ops_for_energy)):
    print(f'Jump operator for energy: {np.round(uniqe_freqs[0][v], 3)}')
    print(jump_ops_for_energy[v])



[[ 0.          +0.j -0.7071067812+0.j  0.          +0.j  0.7071067812+0.j]
 [-0.7071067812+0.j  0.          +0.j  0.7071067812+0.j  0.          +0.j]
 [ 0.          +0.j  0.7071067812+0.j  0.          +0.j  0.7071067812+0.j]
 [ 0.7071067812+0.j  0.          +0.j  0.7071067812+0.j  0.          +0.j]]
Jump operator for energy: -0.45
[[ 0.          +0.j -0.7071067812+0.j  0.          +0.j  0.7071067812+0.j]
 [ 0.          +0.j  0.          +0.j  0.          +0.j  0.          +0.j]
 [ 0.          +0.j  0.          +0.j  0.          +0.j  0.          +0.j]
 [ 0.          +0.j  0.          +0.j  0.          +0.j  0.          +0.j]]
Jump operator for energy: 0.0
[[0.          +0.j 0.          +0.j 0.          +0.j 0.          +0.j]
 [0.          +0.j 0.          +0.j 0.7071067812+0.j 0.          +0.j]
 [0.          +0.j 0.7071067812+0.j 0.          +0.j 0.7071067812+0.j]
 [0.          +0.j 0.          +0.j 0.7071067812+0.j 0.          +0.j]]
Jump operator for energy: 0.45
[[ 0.          +0.j 

In [13]:
# Check if sum gives the whole
# np.allclose(x_jump_in_eigenbasis, np.sum(jump_ops_for_energy, axis=0), atol=1e-12)

In [14]:
# Generate random sparse matrix 
# qubits = 13
# matrix = np.zeros((2**qubits, 2**qubits), dtype=complex)
# matrix[1, 2] = 1
# matrix[2, 1] = 1
# matrix[3, 3] = 1
# matrix[4, 4] = 1
# matrix[5, 10] = 1
# matrix[10, 5] = 1


# #turn to scipy sparse
# sp_matrix = csr_matrix(matrix)

# #random dense numpy matrix
# matrix2 = np.random.rand(2**qubits, 2**qubits) + 1j * np.random.rand(2**qubits, 2**qubits)

# t0 = time.time()
# # multiply the two dense matrices
# np.dot(matrix, matrix2)
# t1 = time.time()
# print('Dense matrix multiplication time: ', t1 - t0)

# sp_matrix2 = csr_matrix(matrix2)
# t0 = time.time()
# # multiply the dense with the sparse matrices
# np.dot(sp_matrix, sp_matrix2)
# t1 = time.time()
# print('Sparse matrix multiplication time: ', t1 - t0)

